In [24]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [25]:
%matplotlib inline

<p> Equação para encontrar magnitude de vento a 10m da superfície <br><b style='color:red'> U10 = Uz*(10/z)^1/7</b> </p>

<p style='font-size:16px; color:black'> funções</p>

In [3]:
def wind10m(wind, height):
    """
    convete um dado de vento
    para a altura de 10m
    """
    w10 = wind*((10/height)**(1/7))
    return w10

<p style="font-size:16px"><h4>importando dados</h4>. 
<br>Aqui serão importados os dados da estação automatica do INMET, este equipamento localiza-se a 29.5m da superfície. Os dados são de velocidade media, rajada e direção.</p>

In [4]:
path = '/home/gus/Documentos/mestrado/dados-mestrado/vento-inmet'

In [26]:
df = pd.read_csv(path+'/vento_bruto.csv', delimiter=';')

In [27]:
df.head(5)

,dates,hours,direction,wg,w
0,2016-12-01,00:00:00,92.0,5.1,1.9
1,2016-12-01,01:00:00,86.0,5.8,2.5
2,2016-12-01,02:00:00,92.0,6.2,2.4
3,2016-12-01,03:00:00,87.0,7.0,2.4
4,2016-12-01,04:00:00,93.0,7.5,3.5


<p style='font-size:20px; color:black'>Encontrando a quatidade de dados invalidos e validos</p>

In [28]:
print(df.isnull().sum())

dates         0
hours         0
direction    13
wg           13
w            13
dtype: int64


In [29]:
df.notna().sum()

dates        9504
hours        9504
direction    9491
wg           9491
w            9491
dtype: int64

<p style='font-size:20px; color:black'>transformando velocidade para 10m</p>

In [30]:
df.w = wind10m(df.w, 29.5)
df.wg = wind10m(df.wg, 29.5)

<p style="font-size:16px"> colocando o dado em matriz sem lacunas de tempo</p>

In [31]:
time_str = df.dates +' '+df.hours

# removendo valores duplicados
time_str = np.unique(time_str).tolist()

<p style='font-size:20px; color:black'>rotacionando os dados e calculando as componentes <b>u</b> e <b>v</b></p>

<p style='font-size:20px; color:black'>realocando os dados</p>

In [32]:
# tempo do anemometro
time = [dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
        for date in time_str]
# del time_str

In [33]:
# vetor de tempo sem lacunas
time_full = pd.date_range(start='2016-12-01 00:00',
                          end='2017-12-31 23:00', freq='1H')

In [34]:
if len(time_full) != len(time_str):
    print('A base de dados possui lacunas temporais.')
    print('Voce precisa realocar os dados em uma matriz sem lacunas de tempo!')

A base de dados possui lacunas temporais.
Voce precisa realocar os dados em uma matriz sem lacunas de tempo!


In [35]:
# removendo colunas inuteis no dataframe
df.drop(['dates','hours'], axis=1, inplace=True)

In [36]:
df.head(3)

,direction,wg,w
0,92.0,4.369711,1.627932
1,86.0,4.969476,2.142015
2,92.0,5.312198,2.056335


In [37]:
rc = np.zeros([len(time_full), 3])
rc.fill(np.NaN)

In [38]:
rc.shape

(9504, 3)

In [39]:
rc

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       ...,
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [40]:
for line in range(len(time)):
    
    idx = np.where(time_full == time[line])
    
    idx = idx[0]
    
    rc[idx, 0] = df.iloc[line, 0]
    rc[idx, 1] = df.iloc[line, 1]
    rc[idx, 2] = df.iloc[line, 2]

del line, idx


<p style='font-size:20px'>Agora os dados de vento foram alocados para uma matriz sem lacunas de tempo. Antes de salvar os dados eu irei colocar eles em um dataframe e adicionar uma coluna com os valores de tempo. </p>

In [41]:
rc = pd.DataFrame(rc)
rc.columns = ['direction', 'wg', 'w']

In [42]:
# correcao para a direcao oceanografia
#direction = df.direction + 180.0
#direction[direction>360] = direction[direction>360]%360.0

In [43]:
#rc['direction'] = direction
#del direction

In [44]:
rc['time'] = time_full

In [45]:
rc.head()

,direction,wg,w,time
0,92.0,4.369711,1.627932,2016-12-01 00:00:00
1,86.0,4.969476,2.142015,2016-12-01 01:00:00
2,92.0,5.312198,2.056335,2016-12-01 02:00:00
3,87.0,5.997643,2.056335,2016-12-01 03:00:00
4,93.0,6.426046,2.998821,2016-12-01 04:00:00


<p style='font-size:16px'> Salvando os dados</p>

In [46]:
rc.to_csv('/home/gus/Documentos/mestrado/dados-mestrado/'+
          'vento-inmet/vento_limpo.csv',sep=';', index = False)